# Start a chat with LLM

The `Branch` object is the foundational building block of lionagi, serving as your primary interface for AI model interactions.

Think of a Branch as a conversation thread that:
- **Manages messages**: Automatically handles system prompts, user inputs, and AI responses in a structured conversation flow
- **Enables multi-turn conversations**: Maintains context across multiple exchanges, allowing for natural back-and-forth dialogue
- **Handles advanced features**: Supports structured outputs, tool usage, and sophisticated context management out of the box

**Architecture Overview:**
- A `Session` can contain multiple branches, but every session starts with a default `'main'` branch
- Each branch maintains its own conversation history in `branch.messages`

This design allows you to organize complex AI workflows into logical conversation threads, each with their own context and purpose.

**Configuration and Model Selection:**

lionagi comes with sensible defaults for popular AI providers. You can explore these configurations in `lionagi.integrations.config` to understand available models and their settings.

The framework supports multiple AI providers through a unified interface, making it easy to switch between different models without changing your core application logic.

### 1. Using `chat()` - Stateless Interactions

The `chat()` method provides **stateless** communication with AI models. Each call is independent, making it perfect for one-off queries or when you want to control conversation history manually.

**When to use `chat()`:**
- Single-shot questions that don't require conversation context
- When you want to manually manage conversation state
- For parallel processing of independent queries

In [1]:
from lionagi import iModel, Branch
from IPython.display import Markdown

In [2]:
SYSTEM = "As a comedian, you are sarcastically funny"
prompt1 = "short joke: a blue whale and a big shark meet at the bar and start dancing"

In [3]:
# create a branch with gpt-4.1-nano model, need an OpenAI API key
comedian1 = Branch(
    chat_model=iModel(model="openai/gpt-4.1-nano"), system=SYSTEM
)
joke1 = await comedian1.chat(prompt1)

Markdown(f"**comedian1**: {joke1}")

**comedian1**: A blue whale and a big shark walk into a bar. They start dancing—turns out, even in the ocean, they're just trying to groove away from their terrible dating profiles.

**Model Provider Flexibility:**

lionagi's unified interface allows you to seamlessly switch between different AI providers. Whether you're using OpenAI's latest models, local Ollama instances, or other providers, the API remains consistent.

This provider-agnostic approach gives you the flexibility to:
- Choose the best model for your specific use case
- Switch providers based on cost, performance, or availability
- Test the same logic across different AI models

### 2. Using `communicate()` - Stateful Conversations

The `communicate()` method provides **stateful** interactions where each message builds upon the previous conversation history. This is ideal for multi-turn dialogues and complex reasoning tasks.

**Key Difference from `chat()`:**
- `chat()` is **stateless**: Each call is independent, no memory between calls
- `communicate()` is **stateful**: Automatically maintains conversation history, enabling context-aware responses

**When to use `communicate()`:**
- Multi-turn conversations where context matters
- Building up complex reasoning over multiple exchanges
- When you want automatic conversation memory management

In [4]:
# Need to install "lionagi[ollama]" to use Ollama models

comedian2 = Branch(
    chat_model=iModel(model="ollama/deepseek-r1:1.5b"), system=SYSTEM
)
joke2 = await comedian2.communicate(prompt1)

Markdown(f"**comedian2**: {joke2}")

**comedian2**: <think>
Okay, so I have this user query. Let's see what they're asking for. They want me to be a comedian who’s sarcastically funny and come up with a joke based on them. The setup is that a blue whale and a big shark meet at the bar and start dancing. Hmm.

Alright, I need to think about how a blue whale and a shark can go smoothly or maybe accidentally dance together in a bar. Well, they're both water-based creatures. That could be a funny angle. Maybe the idea of them being on the water side? 

So, starting with that theme, maybe something about not wanting to be too far from shorelines because there’s a lot of activity when they’re out. That ties into their characteristics. Also, considering the bar itself—sharks can be seen drinking, etc.

Putting it all together, I should avoid sounding too serious and add some humor through their behaviors. Maybe include something playful like swishing arms or bellowing in fits. 

The user also mentioned a short joke, so keeping it concise is key. I need to make sure it's clear and gets the point across without being too wordy.

Alright, let me try formulating that. Maybe something about not wanting to be too far from shorelines because there’s a lot of activities out. That ties into both the blue whale and shark characteristics. 

Then have them start dancing together at the bar, keeping it humorous yet appropriate, adding some flair to make it light despite being the opposite in many ways.
</think>

The blue whale and the big shark aren’t exactly on the same boat when they pop into the bar. They’re both waterberms, like moths trapped under a glass. You know what happens next? They’ll start bellowing in fits and swishing arms like they’ve seen too many fish in total.

In [ ]:
# One message, only the system

comedian1.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,1.753069e+09,system,"{'system_message': 'As a comedian, you are sar...",935c1318-3f5a-4172-b4c4-728bb8948880,system,6e490b3f-7b51-4657-95b6-6e66362338b3,{'lion_class': 'lionagi.protocols.messages.sys...


In [ ]:
# Three messages, system + user + assistant

comedian2.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,1.753069e+09,system,"{'system_message': 'As a comedian, you are sar...",6c39df29-c90e-4a74-a404-7678d6f1f9bd,system,3c5be94e-00b2-46d7-9f0f-7715e768c7c7,{'lion_class': 'lionagi.protocols.messages.sys...
1,1.753069e+09,user,"{'context': [], 'instruction': 'short joke: a ...",41ca3cce-a47e-4496-88c7-fd31a34eefb7,user,3c5be94e-00b2-46d7-9f0f-7715e768c7c7,{'lion_class': 'lionagi.protocols.messages.ins...
2,1.753069e+09,assistant,"{'assistant_response': '<think> Okay, so I hav...",eac70edd-6338-40eb-8525-a8bd25ee5836,3c5be94e-00b2-46d7-9f0f-7715e768c7c7,user,"{'model_response': {'id': 'chatcmpl-198', 'obj..."


In [ ]:
# Review the entire model response from LLMs

comedian2.messages[-1].metadata

{'model_response': {'id': 'chatcmpl-198',
  'object': 'chat.completion',
  'created': 1753069177,
  'model': 'deepseek-r1:1.5b',
  'system_fingerprint': 'fp_ollama',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': "<think>\nOkay, so I have this user query. Let's see what they're asking for. They want me to be a comedian who’s sarcastically funny and come up with a joke based on them. The setup is that a blue whale and a big shark meet at the bar and start dancing. Hmm.\n\nAlright, I need to think about how a blue whale and a shark can go smoothly or maybe accidentally dance together in a bar. Well, they're both water-based creatures. That could be a funny angle. Maybe the idea of them being on the water side? \n\nSo, starting with that theme, maybe something about not wanting to be too far from shorelines because there’s a lot of activity when they’re out. That ties into their characteristics. Also, considering the bar itself—sharks can be seen drinking,